# EXP-003: Kurtosis-Degradation Analysis

**Objective:** Validate that effective kurtosis (κ_eff) predicts quantization sensitivity.

**Hypothesis H2:** r(κ_eff, D) is significant with |r| > 0.7

**Theory:** Per Banner et al. (2019), optimal clipping threshold α* depends on weight distribution kurtosis:
$$\alpha^* = \sigma \cdot (2.5 + 0.3 \cdot \ln(1 + \max(0, \kappa)))$$

**LA-ACIQ Extension:** Different languages activate different layers → different effective κ → different sensitivity.

**References:**
- Banner, R., et al. (2019). "Post-Training 4-bit Quantization." NeurIPS.
- Frühwirth-Schnatter, S. (2006). "Finite Mixture Models." Springer.

In [ ]:
# @title Setup
!pip install -q transformers accelerate bitsandbytes scipy pandas matplotlib seaborn

import torch
import numpy as np
import pandas as pd
from scipy import stats
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import matplotlib.pyplot as plt
import seaborn as sns
from dataclasses import dataclass
from typing import Dict, List, Tuple
import json
import warnings
warnings.filterwarnings('ignore')

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# @title Configuration

MODEL_NAME = "bigscience/bloom-560m"
MAX_LENGTH = 256

# Languages with expected resource levels
LANGUAGES = {
    "en": {"name": "English", "resource": "high"},
    "fr": {"name": "French", "resource": "high"},
    "zh": {"name": "Chinese", "resource": "high"},
    "ar": {"name": "Arabic", "resource": "medium"},
    "he": {"name": "Hebrew", "resource": "low"},
    "sw": {"name": "Swahili", "resource": "low"},
}

# Sample texts
TEXTS = {
    "en": "The study of mathematics reveals the underlying structure of the universe and provides tools for understanding complex phenomena.",
    "fr": "L'étude des mathématiques révèle la structure sous-jacente de l'univers et fournit des outils pour comprendre les phénomènes complexes.",
    "zh": "数学研究揭示了宇宙的基本结构，并为理解复杂现象提供了工具。",
    "ar": "دراسة الرياضيات تكشف البنية الأساسية للكون وتوفر أدوات لفهم الظواهر المعقدة.",
    "he": "לימוד המתמטיקה חושף את המבנה הבסיסי של היקום ומספק כלים להבנת תופעות מורכבות.",
    "sw": "Utafiti wa hisabati unaonyesha muundo wa msingi wa ulimwengu na kutoa zana za kuelewa matukio changamano.",
}

## 1. Load Model and Extract Layer Statistics

In [ ]:
# @title Load Model (FP16 for weight analysis)

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.eval()
print(f"Model loaded. Layers: {model.config.num_hidden_layers}")

In [ ]:
# @title Extract Layer-wise Kurtosis

def extract_layer_kurtosis(model) -> Dict[int, Dict[str, float]]:
    """
    Extract kurtosis statistics for each transformer layer.
    
    Returns dict mapping layer_idx -> {component: kurtosis}
    """
    layer_stats = {}
    
    for name, param in model.named_parameters():
        if "transformer.h." not in name:
            continue
            
        # Parse layer index
        parts = name.split(".")
        try:
            layer_idx = int(parts[2])
        except (IndexError, ValueError):
            continue
        
        if layer_idx not in layer_stats:
            layer_stats[layer_idx] = {"weights": [], "kurtosis": []}
        
        # Compute kurtosis for this parameter
        weights = param.detach().cpu().float().numpy().flatten()
        k = stats.kurtosis(weights, fisher=True)  # Excess kurtosis
        layer_stats[layer_idx]["kurtosis"].append(k)
        layer_stats[layer_idx]["weights"].append(len(weights))
    
    # Compute weighted average kurtosis per layer
    result = {}
    for layer_idx, data in layer_stats.items():
        total_weights = sum(data["weights"])
        weighted_kurtosis = sum(
            k * w / total_weights 
            for k, w in zip(data["kurtosis"], data["weights"])
        )
        result[layer_idx] = {
            "kurtosis": weighted_kurtosis,
            "n_params": total_weights,
        }
    
    return dict(sorted(result.items()))

layer_kurtosis = extract_layer_kurtosis(model)

print("\nLayer Kurtosis (excess):")
for layer, data in layer_kurtosis.items():
    print(f"  Layer {layer:2d}: κ = {data['kurtosis']:7.2f}")

In [ ]:
# @title Identify Outlier Layers

kurtosis_values = [d["kurtosis"] for d in layer_kurtosis.values()]
mean_k = np.mean(kurtosis_values)
std_k = np.std(kurtosis_values)

# Outlier threshold: κ > mean + 2*std
outlier_threshold = mean_k + 2 * std_k

outlier_layers = [
    layer for layer, data in layer_kurtosis.items() 
    if data["kurtosis"] > outlier_threshold
]

print(f"\nKurtosis statistics:")
print(f"  Mean: {mean_k:.2f}")
print(f"  Std: {std_k:.2f}")
print(f"  Max: {max(kurtosis_values):.2f}")
print(f"  Outlier threshold: {outlier_threshold:.2f}")
print(f"  Outlier layers: {outlier_layers}")

## 2. Measure Language-Specific Activations

In [ ]:
# @title Capture Layer Activations

def get_layer_activations(model, tokenizer, text: str) -> Dict[int, float]:
    """
    Capture activation magnitudes per layer for a given text.
    
    Returns: dict mapping layer_idx -> mean activation magnitude
    """
    activations = {}
    hooks = []
    
    def make_hook(layer_idx):
        def hook(module, input, output):
            # output is (hidden_states, ...)
            hidden = output[0] if isinstance(output, tuple) else output
            activations[layer_idx] = hidden.abs().mean().item()
        return hook
    
    # Register hooks
    for i, layer in enumerate(model.transformer.h):
        hooks.append(layer.register_forward_hook(make_hook(i)))
    
    # Forward pass
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=MAX_LENGTH)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        model(**inputs)
    
    # Remove hooks
    for hook in hooks:
        hook.remove()
    
    return activations

# Collect activations for all languages
language_activations = {}
for lang, text in TEXTS.items():
    activations = get_layer_activations(model, tokenizer, text)
    language_activations[lang] = activations
    print(f"{lang}: collected {len(activations)} layer activations")

In [ ]:
# @title Compute Effective Kurtosis per Language

def compute_effective_kurtosis(
    layer_kurtosis: Dict[int, Dict], 
    activations: Dict[int, float]
) -> float:
    """
    Compute effective kurtosis as activation-weighted average.
    
    κ_eff(λ) = Σ_l ā_l(λ) · κ_l
    
    where ā_l is the normalized activation for layer l.
    """
    total_activation = sum(activations.values())
    if total_activation == 0:
        return 0.0
    
    weighted_sum = sum(
        (activations[l] / total_activation) * layer_kurtosis[l]["kurtosis"]
        for l in activations.keys()
        if l in layer_kurtosis
    )
    
    return weighted_sum

# Compute κ_eff for each language
effective_kurtosis = {}
for lang, activations in language_activations.items():
    k_eff = compute_effective_kurtosis(layer_kurtosis, activations)
    effective_kurtosis[lang] = k_eff
    print(f"{lang}: κ_eff = {k_eff:.3f}")

## 3. Measure Quantization Degradation

In [ ]:
# @title Load INT4 Model and Measure Degradation

# Clear cache
del model
torch.cuda.empty_cache()

# Load FP16 for baseline
model_fp16 = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
)
model_fp16.eval()

# Load INT4
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model_int4 = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)
model_int4.eval()

print("Both models loaded.")

In [ ]:
# @title Compute Perplexity and Degradation

def compute_perplexity(model, tokenizer, text: str) -> float:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=MAX_LENGTH)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        return torch.exp(outputs.loss).item()

# Measure for all languages
results = []
for lang, text in TEXTS.items():
    ppl_fp16 = compute_perplexity(model_fp16, tokenizer, text)
    ppl_int4 = compute_perplexity(model_int4, tokenizer, text)
    degradation = (ppl_int4 - ppl_fp16) / ppl_fp16
    
    results.append({
        "lang": lang,
        "name": LANGUAGES[lang]["name"],
        "resource": LANGUAGES[lang]["resource"],
        "k_eff": effective_kurtosis[lang],
        "ppl_fp16": ppl_fp16,
        "ppl_int4": ppl_int4,
        "degradation": degradation,
    })
    
    print(f"{lang}: κ_eff={effective_kurtosis[lang]:.3f}, D={degradation:.4f}")

df = pd.DataFrame(results)

## 4. Hypothesis Testing

In [ ]:
# @title Test H2: Kurtosis-Degradation Correlation

print("="*60)
print("HYPOTHESIS H2: κ_eff correlates with degradation")
print("="*60)

# Pearson correlation
r_pearson, p_pearson = stats.pearsonr(df["k_eff"], df["degradation"])

# Spearman correlation (robust to non-normality)
r_spearman, p_spearman = stats.spearmanr(df["k_eff"], df["degradation"])

print(f"\nPearson correlation:")
print(f"  r = {r_pearson:.4f}")
print(f"  p = {p_pearson:.4f}")

print(f"\nSpearman correlation:")
print(f"  ρ = {r_spearman:.4f}")
print(f"  p = {p_spearman:.4f}")

# Hypothesis evaluation
h2_supported = abs(r_pearson) > 0.7 and p_pearson < 0.05
print(f"\nH2 Result: {'SUPPORTED' if h2_supported else 'NOT SUPPORTED'}")
print(f"  Criterion: |r| > 0.7 and p < 0.05")
print(f"  Observed: |r| = {abs(r_pearson):.4f}, p = {p_pearson:.4f}")

In [ ]:
# @title Visualization

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: Layer kurtosis profile
ax1 = axes[0]
layers = list(layer_kurtosis.keys())
kurtosis = [layer_kurtosis[l]["kurtosis"] for l in layers]
colors = ['red' if l in outlier_layers else 'steelblue' for l in layers]
ax1.bar(layers, kurtosis, color=colors)
ax1.axhline(outlier_threshold, color='red', linestyle='--', label=f'Outlier threshold: {outlier_threshold:.1f}')
ax1.set_xlabel("Layer")
ax1.set_ylabel("Excess Kurtosis")
ax1.set_title("Layer Kurtosis Profile")
ax1.legend()

# Plot 2: κ_eff vs Degradation
ax2 = axes[1]
colors_resource = {"high": "#2ecc71", "medium": "#f39c12", "low": "#e74c3c"}
for _, row in df.iterrows():
    ax2.scatter(row["k_eff"], row["degradation"], 
                c=colors_resource[row["resource"]], s=100, label=row["lang"])
    ax2.annotate(row["lang"], (row["k_eff"], row["degradation"]), 
                 xytext=(5, 5), textcoords='offset points')

# Regression line
z = np.polyfit(df["k_eff"], df["degradation"], 1)
p = np.poly1d(z)
x_line = np.linspace(df["k_eff"].min(), df["k_eff"].max(), 100)
ax2.plot(x_line, p(x_line), 'k--', alpha=0.5)

ax2.set_xlabel("Effective Kurtosis (κ_eff)")
ax2.set_ylabel("Degradation")
ax2.set_title(f"H2: κ_eff vs Degradation (r={r_pearson:.3f})")

# Plot 3: Results table
ax3 = axes[2]
ax3.axis('off')
table_data = df[["lang", "k_eff", "degradation"]].round(4).values
table = ax3.table(
    cellText=table_data,
    colLabels=["Lang", "κ_eff", "Degradation"],
    loc='center',
    cellLoc='center'
)
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.5)
ax3.set_title("Results Summary")

plt.tight_layout()
plt.savefig("exp003_results.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# @title Save Results

results_summary = {
    "experiment": "EXP-003: Kurtosis-Degradation Analysis",
    "model": MODEL_NAME,
    "hypothesis": {
        "H2": {
            "statement": "r(κ_eff, D) significant with |r| > 0.7",
            "r_pearson": round(r_pearson, 4),
            "p_pearson": round(p_pearson, 4),
            "r_spearman": round(r_spearman, 4),
            "p_spearman": round(p_spearman, 4),
            "supported": h2_supported,
        }
    },
    "layer_statistics": {
        "mean_kurtosis": round(mean_k, 4),
        "std_kurtosis": round(std_k, 4),
        "max_kurtosis": round(max(kurtosis_values), 4),
        "outlier_layers": outlier_layers,
    },
    "per_language": df.to_dict(orient="records"),
}

with open("exp003_results.json", "w") as f:
    json.dump(results_summary, f, indent=2)

print("\n" + "="*60)
print("EXPERIMENT COMPLETE")
print("="*60)
print(f"\nH2: {'SUPPORTED' if h2_supported else 'NOT SUPPORTED'}")
print(f"Correlation: r = {r_pearson:.4f} (p = {p_pearson:.4f})")
print(f"\nResults saved to exp003_results.json")

## 5. Interpretation

### Theory Connection

The effective kurtosis κ_eff captures how heavy-tailed the weight distribution "appears" to a given language based on its activation pattern.

**If correlation is NEGATIVE (r < 0):**
- Higher κ_eff → LESS degradation
- This makes sense: Banner's formula gives larger α* for higher κ
- Global α* is tuned for average κ
- Languages with above-average κ_eff benefit (less clipping)
- Languages with below-average κ_eff suffer (too much clipping)

**If correlation is POSITIVE (r > 0):**
- Higher κ_eff → MORE degradation
- This would contradict the simple theory
- May indicate other factors dominate

### Limitations

- Small sample (6 languages)
- Single text per language
- κ_eff is approximation (activation-weighted, not true mixture)